In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')

칼럼 생성

In [ ]:
df['년'] = df['일시'].str[2:4]
df['월'] = df['일시'].str[5:7]
df['일'] = df['일시'].str[8:10]

결측치

In [ ]:
# 3. '강수량(mm)' 칼럼에서 값이 0인 것을 0.1로 변경
df['인천_강수량(mm)'] = df['인천_강수량(mm)'].replace(0, 0.1)

# 4. '강수량(mm)' 칼럼에서 결측치를 0으로 변경
df['인천_강수량(mm)'] = df['인천_강수량(mm)'].fillna(0)

# 5. 나머지 칼럼들('이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '중하층운량(10분위)')의 결측치를 보간법으로 채우기
#df[['기온(°C)','풍속(m/s)','풍향(16방위)','습도(%)','이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '중하층운량(10분위)']] = df[['기온(°C)','풍속(m/s)','풍향(16방위)','습도(%)','이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '중하층운량(10분위)']].interpolate()

행 제거

In [ ]:
# 조건 1: 월 칼럼이 '05'이고 일 칼럼이 '31'인 경우
condition1 = (df['월'] == '05') & (df['일'] == '31')

# 조건 2: 월 칼럼이 '06', '07', '08', '09' 중 하나인 경우
condition2 = df['월'].isin(['06', '07', '08', '09'])

# 조건 3: 월 칼럼이 '10'이고 일 칼럼이 '01'인 경우
condition3 = (df['월'] == '10') & (df['일'] == '01')

# 위의 조건 중 하나라도 만족하는 행을 남김
df = df[condition1 | condition2 | condition3]

결측치 확인

In [ ]:
df.isnull().sum()

,0
지점명,0
일시,0
서울_강수량(mm),0
서울_풍속(m/s),0
서울_풍향(16방위),0
서울_습도(%),0
서울_이슬점온도(°C),0
서울_현지기압(hPa),0
서울_중하층운량(10분위),0
수원_강수량(mm),0


DF 전체 / 생략

In [ ]:
pd.reset_option('display.max_rows')


In [ ]:
pd.set_option('display.max_rows', None)

일별 결측치

In [ ]:
# 월, 일 칼럼별로 행의 개수를 세기
grouped_counts = df.groupby(['년', '월', '일']).size().reset_index(name='행 개수')

# 행 개수가 24이 아닌 경우만 필터링
filtered_counts = grouped_counts[grouped_counts['행 개수'] != 24]

filtered_counts

,년,월,일,행 개수


파일

In [ ]:
df.to_csv('/content/drive/MyDrive/강수/기후24.csv', encoding = 'cp949')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/강수/기후24.csv', encoding = 'cp949')

LAG

In [ ]:
# 강수량 칼럼들 리스트
rain_columns = ['서울_강수량(mm)', '수원_강수량(mm)', '서산_강수량(mm)', '인천_강수량(mm)']

# 각 강수량 칼럼에 대해 1시간부터 3시간까지의 lag 변수 생성
for col in rain_columns:
    for lag in range(1, 4):  # 1시간부터 3시간까지
        df[f'{col}_{lag}시간전'] = df[col].shift(lag)

In [ ]:
# 강수량 칼럼들 리스트
rain_columns = ['서울_강수량(mm)']

# 각 강수량 칼럼에 대해 미래 변수 생성
for col in rain_columns:
    for lead in range(1, 4):  # 1시간부터 3시간까지
        df[f'{col}_{lead}시간후'] = df[col].shift(-lead)

In [ ]:
df

,지점명,일시,서울_강수량(mm),서울_풍속(m/s),서울_풍향(16방위),서울_습도(%),서울_이슬점온도(°C),서울_현지기압(hPa),서울_중하층운량(10분위),수원_강수량(mm),...,수원_강수량(mm)_3시간전,서산_강수량(mm)_1시간전,서산_강수량(mm)_2시간전,서산_강수량(mm)_3시간전,인천_강수량(mm)_1시간전,인천_강수량(mm)_2시간전,인천_강수량(mm)_3시간전,서울_강수량(mm)_1시간후,서울_강수량(mm)_2시간후,서울_강수량(mm)_3시간후
0,서울,2024-05-31 1:00,0.0,1.0,270.0,61.0,11.8,1000.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,서울,2024-05-31 2:00,0.0,1.3,340.0,60.0,11.5,999.4,9.0,0.0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0
2,서울,2024-05-31 3:00,0.0,2.0,290.0,65.0,12.2,999.3,2.0,0.0,...,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0
3,서울,2024-05-31 4:00,0.0,1.9,290.0,68.0,11.9,999.4,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,서울,2024-05-31 5:00,0.0,1.8,290.0,72.0,12.2,999.7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,서울,2024-08-13 20:00,0.0,3.5,320.0,66.0,24.0,997.3,0.0,0.0,...,18.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1796,서울,2024-08-13 21:00,0.0,1.2,320.0,70.0,24.6,997.9,0.0,0.0,...,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1797,서울,2024-08-13 22:00,0.0,1.2,20.0,73.0,25.0,998.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1798,서울,2024-08-13 23:00,0.0,1.4,320.0,75.0,25.1,997.9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


VIF

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 숫자형 데이터만 선택
X = df.select_dtypes(include=[np.number])

# 결측치와 무한대 값 처리
X = X.dropna()  # 결측치가 있는 행을 제거
X = X[np.isfinite(X).all(1)]  # 무한대 값이 있는 행을 제거

# VIF 데이터프레임 생성
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF 출력
vif_data

,Feature,VIF
0,년,11.697209
1,월,69.967548
2,일,4.233861
3,이산화질소농도(ppm),14.913000
4,오존농도(ppm),10.117630
5,아황산가스농도(ppm),7.382021
6,미세먼지농도(㎍/㎥),5.650737
7,서울_장마,1.773453
8,서울_기온(°C),2604.733459
9,서울_강수량(mm),1.676318


CORR

In [ ]:
# 일시 칼럼을 제외하고 상관계수 계산
correlation_matrix = df.drop(columns=['일시']).corr()

# 서울_강수량(mm)과 다른 칼럼들의 상관계수 추출
seoul_rain_corr = correlation_matrix['서울_습도(%)']

seoul_rain_corr

,서울_습도(%)
년,0.000525
월,0.019272
일,0.032663
이산화질소농도(ppm),-0.106068
오존농도(ppm),-0.251534
아황산가스농도(ppm),-0.125425
미세먼지농도(㎍/㎥),-0.030887
서울_장마,0.220924
서울_기온(°C),-0.405132
서울_강수량(mm),0.227581


PCA

In [ ]:
from sklearn.decomposition import PCA

# 서울, 수원, 서산, 인천 각각의 칼럼 이름 리스트
columns_seoul = ["서울_습도(%)", "서울_이슬점온도(°C)", "서울_기온(°C)"]
columns_suwon = ["수원_습도(%)", "수원_이슬점온도(°C)", "수원_기온(°C)"]
columns_seosan = ["서산_습도(%)", "서산_이슬점온도(°C)", "서산_기온(°C)"]
columns_incheon = ["인천_습도(%)", "인천_이슬점온도(°C)", "인천_기온(°C)"]

# PCA 객체 생성 (주성분 수는 2로 설정)
pca = PCA(n_components=1)

# 각 지역별로 PCA 적용 및 원래 칼럼 삭제
df_seoul_pca = pca.fit_transform(df[columns_seoul])
df_suwon_pca = pca.fit_transform(df[columns_suwon])
df_seosan_pca = pca.fit_transform(df[columns_seosan])
df_incheon_pca = pca.fit_transform(df[columns_incheon])

# 결과를 데이터프레임으로 변환하여 보기 좋게 이름을 지정
df_seoul_pca = pd.DataFrame(df_seoul_pca, columns=["서울_PC1"])
df_suwon_pca = pd.DataFrame(df_suwon_pca, columns=["수원_PC1"])
df_seosan_pca = pd.DataFrame(df_seosan_pca, columns=["서산_PC1"])
df_incheon_pca = pd.DataFrame(df_incheon_pca, columns=["인천_PC1"])

# PCA 대상 칼럼을 데이터프레임에서 삭제
df = df.drop(columns=columns_seoul + columns_suwon + columns_seosan + columns_incheon)

# 각 PCA 결과를 원래 데이터프레임에 병합
df = pd.concat([df, df_seoul_pca, df_suwon_pca, df_seosan_pca, df_incheon_pca], axis=1)

In [ ]:
df

,일시,년,월,일,이산화질소농도(ppm),오존농도(ppm),아황산가스농도(ppm),미세먼지농도(㎍/㎥),서울_장마,서울_강수량(mm),...,인천_강수량(mm),인천_풍속(m/s),인천_풍향(16방위),인천_현지기압(hPa),인천_해면기압(hPa),인천_중하층운량(10분위),서울_PC1,수원_PC1,서산_PC1,인천_PC1
0,2000-05-31 0:00,0,5,31,0.0340,0.0290,0.0030,21.0,0,0.0,...,0.0,0.8,320.0,998.700000,1006.600000,6.000000,2.200229,3.646223,3.062474,-1.485848
1,2000-05-31 1:00,0,5,31,0.0340,0.0290,0.0030,21.0,0,0.1,...,0.0,1.9,320.0,998.100000,1006.033333,6.333333,4.108052,2.631542,1.028048,-5.462297
2,2000-05-31 2:00,0,5,31,0.0340,0.0290,0.0030,21.0,0,0.0,...,0.0,1.2,340.0,997.500000,1005.466667,6.666667,4.068466,1.626786,-1.006378,-5.489602
3,2000-05-31 3:00,0,5,31,0.0340,0.0290,0.0030,21.0,0,0.0,...,0.0,1.6,320.0,996.900000,1004.900000,7.000000,-0.839642,2.623408,-6.013576,-3.536413
4,2000-05-31 4:00,0,5,31,0.0340,0.0290,0.0030,21.0,0,0.0,...,0.0,0.3,20.0,996.733333,1004.733333,7.000000,1.073569,-1.416399,-6.044108,0.383896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71419,2023-10-01 19:00,23,10,1,0.0058,0.0358,0.0035,23.0,0,0.0,...,0.0,3.4,320.0,1006.800000,1014.800000,0.000000,9.179123,5.617711,7.118966,35.083776
71420,2023-10-01 20:00,23,10,1,0.0058,0.0358,0.0035,23.0,0,0.0,...,0.0,3.0,320.0,1007.200000,1015.200000,0.000000,11.275420,1.585181,0.037599,38.277721
71421,2023-10-01 21:00,23,10,1,0.0058,0.0358,0.0035,23.0,0,0.0,...,0.0,2.4,320.0,1008.000000,1016.000000,0.000000,20.557888,-6.470887,-7.021360,33.153295
71422,2023-10-01 22:00,23,10,1,0.0058,0.0358,0.0035,23.0,0,0.0,...,0.0,3.4,320.0,1008.300000,1016.400000,0.000000,18.536475,-12.491250,-12.048538,30.082881
